In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import regex as re
from bs4 import BeautifulSoup
import requests
import urllib
from urllib.request import urlopen
from selenium.common.exceptions import NoSuchElementException
from collections import Counter
import math
import io
from PyPDF2 import PdfReader
import datefinder
import plotly.express as px
import plotly.graph_objects as go

c:\users\ramon\appdata\local\programs\python\python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.1.0)/charset_normalizer (2.0.11) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
def check_timeframes(timeframes):
    
    Correct_Date = True
    
    for timeframe in timeframes:
        if len(timeframe) != 16:
            Correct_Date = False       
        if timeframe[4] != ' ':
            Correct_Date = False
        if timeframe[-10:] != 'e kwartaal':
            Correct_Date = False
        if not re.findall('(:19[7-9]\d|2\d{3})(?=\D|$)', timeframe[0:4]):
            Correct_Date = False
        if not timeframe[5] in ['1','2','3','4']:
            Correct_Date = False
            
    return Correct_Date

def data_subset(data, start_time, end_time):  
    
    data['kwartaal_int'] = data['kwartaal'].astype(str).str.slice(0 , 1)

    cols = data.columns

    min_jaar = int(start_time[0:4])
    max_jaar = int(end_time[0:4])

    min_kwartaal = start_time[5]
    max_kwartaal = end_time[5]

    rows = []

    for i, r in data.iterrows():

        if r.jaar < min_jaar or r.jaar > max_jaar:
            pass

        elif min_jaar == max_jaar:
            if r.kwartaal_int >= min_kwartaal and r.kwartaal_int <= max_kwartaal:
                rows.append(list(r))

        else:
            if r.jaar == min_jaar:
                if min_kwartaal > r.kwartaal_int:
                    pass
                else:
                    rows.append(list(r))
            if r.jaar == max_jaar:
                if max_kwartaal < r.kwartaal_int:
                    pass
                else:
                    rows.append(list(r))        

    data = pd.DataFrame(data = rows, columns = cols)
    data = data.drop('kwartaal_int', axis=1)

    return data

In [4]:
def maak_plots(start_time = '2022 1e Kwartaal', end_time = '2023 1e Kwartaal'):
    
    data = pd.read_csv('PlotData.csv')
    timeframes = [start_time.lower(), end_time.lower()]
    
    if check_timeframes(timeframes) == False:
        data = data
    
    else:
        data = data_subset(data, start_time, end_time)

    display(data)
        
    ## TODO maken van plots hier
        
    
print(maak_plots('2022 1e Kwartaal', '2023 1e Kwartaal'))

,Verzoek,docdatums,urls,verdatums,verantwoordelijke,jaar,kwartaal,dagen
0,Beslissing op bezwaar Wob-/Woo-verzoek over in...,2022-12-23,['https://open.overheid.nl/documenten/ronl-7a0...,2022-07-14,Ministerie van Infrastructuur en Waterstaat,2022,4e Kwartaal,162
1,Beslissing op bezwaar Wob-/Woo-verzoek over ve...,2023-01-27,['https://open.overheid.nl/documenten/ronl-0ba...,2021-11-02,"Ministerie van Landbouw, Natuur en Voedselkwal...",2023,1e Kwartaal,451
2,Beslissing op bezwaar Wob-besluit Financiën ov...,2022-04-26,['https://open.overheid.nl/documenten/ronl-e3e...,2021-10-22,Ministerie van Financiën,2022,2e Kwartaal,186
3,Beslissing op bezwaar Wob-besluit over afstemm...,2023-02-09,['https://open.overheid.nl/documenten/ronl-07e...,2022-10-05,Ministerie van Financiën,2023,1e Kwartaal,127
4,Beslissing op bezwaar Wob-besluit over contact...,2023-02-08,['https://open.overheid.nl/documenten/ronl-aa6...,2021-03-25,Ministerie van Financiën,2023,1e Kwartaal,685
...,...,...,...,...,...,...,...,...
818,Besluit over koude weer training,2022-09-13,['https://open.overheid.nl/documenten/ronl-60d...,2022-06-09,Ministerie van Defensie,2022,3e Kwartaal,96
819,Woo-verzoek Evaluatiecommissie Tuitjenhorn,2022-06-30,['https://open.overheid.nl/documenten/ronl-370...,2021-10-22,Ministerie van Justitie en Veiligheid,2022,2e Kwartaal,251
820,Woo-verzoek over de identiteit achter anonieme...,2022-08-25,['https://open.overheid.nl/documenten/ronl-2a0...,2022-08-02,"Ministerie van Landbouw, Natuur en Voedselkwal...",2022,3e Kwartaal,23
821,Woo-verzoek over landbouwvrijstelling in inkom...,2023-02-27,['https://open.overheid.nl/documenten/ronl-1c0...,2022-08-07,Ministerie van Financiën,2023,1e Kwartaal,204


None
